In [ ]:
import py4dgeo
import numpy as np
from scipy.spatial import KDTree as ScipyKDTree
from sklearn.neighbors import KDTree as SklearnKDTree
import laspy
from time import perf_counter

In [ ]:
def measure(f):
    """ A decorator that measures execution time and returns it as part of a tuple """
    def _decorated(*args, **kwargs):
        start = perf_counter()
        ret = f(*args, **kwargs)
        return perf_counter() - start, ret
    return _decorated

In [ ]:
def create_random_data(n):
    """ Create n samples within the unitcube """
    rng = np.random.default_rng()
    return rng.uniform([0, 0, 0], [1, 1, 1], size=(n, 3))

In [ ]:
def read_huge_las_dataset():
    """ Read 250M points dataset """
    data = laspy.read("ahk_2017_full.las")
    return np.stack((data.x.array, data.y.array, data.z.array), axis=1)

In [ ]:
def read_small_las_dataset():
    """ Read 2M points dataset """
    return np.genfromtxt('ahk_2017_small.xyz', delimiter=' ', dtype=np.float32)

In [ ]:
@measure
def build_pcl_kdtree(data):
    """ Build PCL KDTree data structure. """
    pc = py4dgeo.PCLPointCloud(data)
    pc.build_tree(py4dgeo.SearchStrategy.kdtree)
    return pc

In [ ]:
@measure
def build_pcl_bruteforce(data):
    """ Build PCL Bruteforce data structure. """
    pc = py4dgeo.PCLPointCloud(data)
    pc.build_tree(py4dgeo.SearchStrategy.bruteforce)
    return pc

In [ ]:
@measure
def build_pcl_octree(data):
    """ Build PCL OCTree data structure. """
    pc = py4dgeo.PCLPointCloud(data)
    pc.build_tree(py4dgeo.SearchStrategy.octree)
    return pc

In [ ]:
@measure
def radius_pcl(tree, point, radius):
    """ Invocation of radius search for PCL trees """
    return tree.radius_search(point, radius)

In [ ]:
@measure
def build_scipy(data):
    """ Build Scipy KDTree data structure """
    return ScipyKDTree(data)

In [ ]:
@measure
def radius_scipy(tree, point, radius):
    """ Invocation of Scipy radius search """
    return tree.query_ball_point(point, radius)

In [ ]:
@measure
def build_sklearn(data):
    """ Build Sklearn KDTree data structure """
    return SklearnKDTree(data)

In [ ]:
@measure
def radius_sklearn(tree, point, radius):
    """ Invocation of Sklearn radius search """
    return tree.query_radius(np.expand_dims(point, axis=0), radius)

In [ ]:
# Make the different implementations iterable
implementations = [
    (build_pcl_kdtree, radius_pcl),
    (build_pcl_bruteforce, radius_pcl),
    (build_pcl_octree, radius_pcl),
    (build_scipy, radius_scipy),
    (build_sklearn, radius_sklearn)
]